In [37]:
import numpy as np
import pandas as pd
import spacy
import nltk 
import requests
import json
import googletrans
import random

from random import randint
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import wordnet 
from googletrans import Translator

spacy.load('en_core_web_sm')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

analyser = SentimentIntensityAnalyzer()
parser = English()
porter = PorterStemmer()
translator = Translator()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ninagroot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
file = 'edit-db_model2.xlsx'
df_gratitude = pd.read_excel(file, 'gratitude')
#df_exercises = pd.read_excel('exercises_model2.xlsx')
df_journal = pd.read_excel(file, 'sc_journal')

topics = {
    'social media': ['social', 'media', 'network', 'twitter', 'facebook', 'instagram', 'web', 'linkedin', 'google', 'viral', 'google+', 'event', 'website', 'reach', 'telegraph', 'telephone', 'blog', 'internet', 'social', 'sociality', 'technoself', 'intranet', 'cyber', 'business', 'sociable', 'activist', 'microblogging', 'socially', 'cybernetic', 'pinterest', 'youtube', 'tumblr', 'cyberspace', 'extranet', 'telnet', 'cybernetics', 'modem', 'communication', 'informatics', 'virtual', 'party', 'picture'],
    'body insecurity': ['body insecurity', 'body', 'insecure', 'belly', 'skin', 'waist', 'hair', 'eyes', 'lips', 'ears', 'forehead', 'eyebrow', 'chin', 'scar', 'nose', 'look', 'eat', 'ate', 'boldness', 'beard', 'smile', 'teeth', 'stomach', 'back', 'ass', 'butt', 'frame', 'feature', 'physique', 'bod', 'bodies', 'torso', 'arm', 'head', 'corpse', 'chest', 'shape', 'shoulder', 'form', 'neck', 'structure', 'leg', 'mass', 'weight', 'shaky', 'unattractive', 'unsure', 'anxious', 'jealous', 'unconfident', 'distrustful', 'uncomfortable', 'unsecured', 'neurotic', 'hopeless', 'depressed', 'introverted', 'helpless', 'fragile', 'danger', 'anxiety', 'mistrust', 'instability', 'unrest', 'hardship', 'vulnerability', 'lawlessness', 'uncertainty', 'turmoil', 'strife', 'fragility', 'hopelessness', 'malnutrition', 'alienation', 'violence', 'poverty', 'tension', 'shortages', 'despair', 'chaos', 'distrust', 'resentment', 'underdevelopment', 'anarchy', 'paranoia', 'cynicism', 'fear', 'scared', 'insecure', 'fearful'],
    'social anxiety': ['social anxiety', 'shyness', 'blushing', 'fear', 'worry', 'stress', 'nervousness', 'attention', 'shame', 'panic', 'social', 'attack', 'own', 'activities', 'lonely', 'alone', 'rejection', 'public', 'speaking', 'nausea', 'stutter', 'acting', 'performance', 'stage', 'fright', 'fear', 'public', 'shyness', 'blushing', 'anxiety', 'self-consciousness', 'panic', 'illness', 'mood disorder', 'timidness', 'social functioning', 'parties', 'outgoing', 'illness', 'peer rejection', 'stutter', 'stage fright', 'guilt', 'panic attack'],
    'relationships' : ['friendship', 'best', 'friend', 'classmate', 'schoolmate', 'roomie', 'boyfriend', 'sister', 'amigo', 'acquanitance', 'brother', 'comerade', 'girlfriend', 'person', 'pal', 'schoolfriend', 'mate', 'colleague', 'roommate', 'love', 'pal', 'buddy', 'talk', 'soulmate', 'fight', 'trust', 'understanding', 'happiness', 'relations', 'empathy', 'company', 'feelings', 'unity', 'friendship', 'friend', 'companionship', 'relationship', 'relation', 'friendship', 'affinity', 'kinship', 'affiliation', 'association', 'partnership', 'acquaintance', 'state', 'consanguinity', 'sisterhood', 'brotherhood', 'membership', 'human relationship', 'family relationship', 'relate', 'relational', 'interrelation', 'family', 'partner', 'relatedness', 'relatedness', 'correlate', 'parentage', 'relative', 'tie', 'account', 'involvement', 'ties', 'affair', 'engagement', 'contacts', 'rapport', 'connection', 'dealing', 'terms', 'regards', 'concerned', 'comparison', 'linkage', 'motherhood', 'maternity', 'paternity', 'fatherhood', 'subjection', 'birth', 'romance', 'life', 'understanding', 'shared', 'future', 'complicated', 'marriage', 'ally', 'feelings', 'intereseted', 'personality', 'mutual', 'personal', 'incest', 'foster', 'absence', 'bilateral', 'sense', 'resolve', 'cousin', 'business relationship', 'love affair', 'personal relation', 'friendly relationship', 'blood kinship', 'marital bed', 'line of descent', 'sexual relationship', 'marital relationship', 'phylogenetic relationship', 'aunt', 'familial', 'fraternal', 'sister', 'nephew', 'grandniece', 'analogy', 'siblinghood', 'framily', 'brother', 'heteronymous', 'neighbourship', 'granddaughter', 'nieceship', 'niece', 'kiss cousin', 'social relation', 'stepbrother', 'intimacy', 'uncle', 'spousal', 'stepsister', 'familywise', 'hookup', 'nonmarriage', 'private life', 'stepfamily', 'interaction', 'lust', 'platonic love', 'affection', 'kindness', 'pleasure', 'compassion', 'sex', 'hate', 'heart', 'emotion', 'religion', 'sexual attraction', 'passion', 'lover', 'fight', 'lie'],
    'work': ['work', 'job', 'turn', 'ferment', 'exercise', 'play', 'sour', 'work out', 'labor', 'go', 'employment', 'put to work', 'process', 'bring', 'workplace', 'toil', 'coursework', 'forge', 'make', 'labour', 'housewifery', 'service', 'exchange', 'studio', 'answer', 'workpiece', 'spadework', 'ironwork', 'leatherwork', 'lacework', 'employ', 'metalwork', 'paperwork', 'nightwork', 'writing', 'welfare work', 'lacquerware', 'learning', 'masterpiece', 'silverwork', 'task', 'workload', 'crop', 'run', 'operate', 'function', 'wreak', 'work on', 'act', 'puzzle out', 'make for', 'piece of work', 'do work', 'project', 'deal', 'work ethic', 'deal', 'exploit', 'overwork', 'worker', 'housekeeping', 'business', 'task', 'employment', 'vocation', 'money', 'subcontract', 'profession', 'career', 'employer', 'place', 'duty', 'workplace', 'hire', 'internship', 'problem', 'chore', 'volunteer', 'position', 'laborer', 'handyman', 'occupation', 'application', 'appointment', 'computing'], 
    'school' : ['school', 'education', 'university', 'academy', 'college', 'teacher', 'classroom', 'grammar school', 'student', 'primary school', 'secondary school', 'educate', 'vocational school', 'seminary', 'schoolhouse', 'kindergarten', 'educational institution', 'conservatory', 'day school', 'shoal', 'public school', 'elementary school', 'private school', 'middle school', 'graduate school', 'curriculum', 'academic', 'gymnasium', 'schoolteacher', 'lyceum', 'deconstructivism', 'schoolroom', 'institution', 'survey', 'learn', 'read', 'research', 'report', 'analyze', 'compary', 'essay', 'major', 'memorizer', 'science', 'learning', 'examination', 'graduation', 'review', 'concentration', 'memorise', 'discipline', 'field of study', 'subject field', 'subject area', 'graduate', 'knowledge', 'researchers', 'analysis', 'schoolwork', 'preparation', 'homework', 'coursework', 'tutoring', 'typing', 'teaching', 'prep', 'lesson', 'tutorial', 'basics', 'housework', 'kids', 'math', 'errands', 'assignments', 'preschool', 'tasks', 'exams', 'paperwork', 'mathematics', 'english', 'history', 'literature', 'thesis', 'fail', 'delay', 'language', 'geography', 'course' ]
    }


In [1]:
df_journal = df_journal.dropna(subset = ['id'])
df_journal_lang = df_journal
df_journal_lang['description_split'] = df_journal_lang['description'].str.rsplit()
df_journal_lang['length'] = df_journal_lang.description_split.str.len() 
df_journal_lang = df_journal_lang[df_journal_lang['length'] > 20]  

NameError: name 'df_journal' is not defined

In [40]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def lemma1(word):
    return WordNetLemmatizer().lemmatize(word)

def lemma2(word):
    return WordNetLemmatizer().lemmatize(word,'v')

def prepare_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 2] #check what works best 
    #tokens = [porter.stem(token) for token in tokens] #stemming
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens] #lemmatization 
    tokens = [lemma2(token) for token in tokens] #lemmatization verbs
    return tokens

def prepare_topics(topics):
    for topic in topics: 
        prepared_words = [] 
        for string in topics[topic]:
            string = prepare_text(string)
            prepared_words.append(string)
            res = []
            [res.append(x) for x in prepared_words if x not in res] 
            res = [' '.join(map(str, i)) for i in res]
        topics[topic] = res
    return topics 

def sentiment_analyzer(sentence):
    score = analyser.polarity_scores(sentence)
    negative_score = score['neg']
    positive_score = score['pos']
    neutral_score = score['neu']
    difference = abs(negative_score - positive_score)
    if neutral_score > 0.8:
        return 'neutral'
    elif difference <0.1:
        return 'neutral'
    elif negative_score > positive_score:
        return 'negative'
    else:
        return 'positive'

def dutch_translator(text):
    result = translator.translate(text, dest='en',  src='nl')
    return result.text

def eng_translator(text):
    result = translator.translate(text, dest='nl',  src='en')
    return result.text

In [2]:
df_journal_lang['description_eng'] = df_journal_lang.apply(lambda row: dutch_translator(row.description), axis = 1)
df_journal_lang['tag'] = df_journal_lang.apply(lambda row: dutch_translator(row.tag), axis = 1)
df_journal_lang['prepared_description'] = df_journal_lang.apply(lambda row: prepare_text(row.description_eng), axis = 1)
df_journal_lang = df_journal_lang.drop(['timestamp', 'rating', 'sc_description', 'length', 'description_split'], axis = 'columns')

NameError: name 'df_journal_lang' is not defined

In [42]:
df_journal_lang.to_excel('prepared_journal_model2_lang.xlsx')

In [3]:
prepared_topics = prepare_topics(topics)
df_journal['description_eng'] = df_journal.apply(lambda row: dutch_translator(row.description), axis = 1)
df_journal['prepared_description'] = df_journal.apply(lambda row: prepare_text(row.description_eng), axis = 1)
df_journal = df_journal.drop(['timestamp', 'rating', 'sc_description', 'length', 'description_split'], axis = 'columns')
display(df_journal)

NameError: name 'prepare_topics' is not defined

In [22]:
df_journal.to_excel('prepared_journal_model2.xlsx')